<a href="https://colab.research.google.com/github/RubensBBatista/ri_labs/blob/master/lab06/modelo_vetorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [100]:
import pandas as pnd
import numpy as nump
import nltk
import re
import collections
import bisect
from nltk.tokenize import RegexpTokenizer

nltk.download('stopwords')
resultado = pnd.read_csv('results.csv')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rubens.sousa\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## 1. Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade. Se você já estiver usando esses dados não precisa reconstruir o índice;

In [101]:
tokniz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
indices = {}
n = 0
stopw = nltk.corpus.stopwords.words('portuguese') 

for texto in resultado.text:
  palavras = [palavra for palavra in tokniz.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopw and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elemento in indices.items():
  dic = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(dic.items())

In [102]:
indices_dfr = pnd.DataFrame()
indices_dfr['Palavra'] = indices.keys()
indices_dfr['Documentos e Frequencias'] = indices.values()
indices_dfr.head(10)

,Palavra,Documentos e Frequencias
0,juíza,"[(1, 2), (2, 1)]"
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ..."
2,ivani,"[(1, 1), (2, 1)]"
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73..."
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78..."
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4..."
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1)]"
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (..."
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1)]"
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,..."


## 2. Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário; 

In [103]:
M = resultado.text.count()
for palavra in indices:
  k = len(indices[palavra])
  aux = nump.log((M+1)/k)
  IDF = round(aux,2)
  indices[palavra].append(IDF)

In [104]:
idf = []
for index in indices.values():
  idf.append(index[-1])
indices_dfr['IDF'] = idf

indices_dfr.head(10)

,Palavra,Documentos e Frequencias,IDF
0,juíza,"[(1, 2), (2, 1), 4.83]",4.83
1,federal,"[(1, 2), (2, 2), (3, 1), (6, 1), (7, 3), (15, ...",1.63
2,ivani,"[(1, 1), (2, 1), 4.83]",4.83
3,silva,"[(1, 3), (2, 1), (6, 1), (14, 2), (26, 1), (73...",3.04
4,luz,"[(1, 3), (2, 1), (9, 1), (17, 1), (32, 2), (78...",2.53
5,brasília,"[(1, 1), (8, 1), (33, 1), (35, 1), (44, 1), (4...",2.19
6,proibiu,"[(1, 1), (2, 1), (119, 1), (162, 1), 4.14]",4.14
7,caráter,"[(1, 1), (15, 1), (36, 1), (60, 1), (89, 1), (...",3.22
8,liminar,"[(1, 1), (2, 3), (119, 1), (217, 1), 4.14]",4.14
9,nesta,"[(1, 2), (3, 1), (4, 1), (8, 1), (21, 1), (22,...",0.91


## 3. Implementar as seguintes versões do modelo vetorial:

### 3.1. Representação binária

In [105]:
def vm_binario(consulta, documento):
  pontos = 0
  tokens_consulta = consulta.split()
  documentos_consulta = documento.split()
  
  for token in tokens_consulta:
    pontos += (token in documentos_consulta)
    
  return pontos

### 3.2. TF (lembre-se que esse modelo já está implementado)

In [106]:
def vm_tf(consulta, documento):
  pontos = 0
  documentos_consulta = documento.split()
  tokens_consulta = consulta.split()
  
  for word in tokens_consulta:
    pontos += documentos_consulta.count(word)
  
  return pontos

### 3.3 TF-IDF

In [107]:
def vm_tf_idf(consulta, documento):
  pontos = 0
  token_documentos = documento.split()
  consulta_tokens = consulta.split()
  
  for palavra in consulta_tokens:
    cwd = token_documentos.count(palavra)
    if palavra in indices:
      pontos += cwd * indices[palavra][-1]
  
  return round(pontos,2)

### 3.4 BM25 (não usaremos Okapi já que os documentos não tem grande variação de tamanho)

In [108]:
def vm_bm25(consulta, documento, k):
  pontos = 0
  documento_tokens = documento.split()
  consulta_tokens = consulta.split()
  
  palavras = [palavra for palavra in consulta_tokens if palavra in documento_tokens]
    
  for palavra in palavras:
    cwd = documento_tokens.count(palavra)
    dfw = 0
    if palavra in indices:
      dfw = len(indices[palavra][:-1])
    pontos += (((k+1) * cwd) / (cwd + k)) * nump.log10(((M+1) / dfw)) if dfw != 0 else 0
  
  return round(pontos,2)

## 4. Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta;

In [109]:
consultas = ['economia', 'neymar', 'bolsonaro']

In [110]:
def top5_modelos(consulta):
  n = 0
  d_b = []
  d_tf = []
  d_tf_idf = []
  d_bm25 = []
  for documento in resultado.text:
    documento = documento.lower()
    n += 1
    bisect.insort(d_b, (vm_binario(consulta, documento), n))
    bisect.insort(d_tf, (vm_tf(consulta,documento), n))
    bisect.insort(d_tf_idf, (vm_tf_idf(consulta,documento), n))
    bisect.insort(d_bm25, (vm_bm25(consulta,documento,20), n))
  
  d_b.reverse()
  d_tf.reverse()
  d_tf_idf.reverse()
  d_bm25.reverse()
  
  return d_b[:5], d_tf[:5], d_tf_idf[:5], d_bm25[:5]

In [111]:
top5_binario = ['','','']
top5_tf = ['','','']
top5_tfidf = ['','','']
top5_bm25 = ['','','']

top5_binario[0], top5_tf[0], top5_tfidf[0], top5_bm25[0] = top5_modelos(consultas[0])
top5_binario[1], top5_tf[1], top5_tfidf[1], top5_bm25[1] = top5_modelos(consultas[1])
top5_binario[2], top5_tf[2], top5_tfidf[2], top5_bm25[2] = top5_modelos(consultas[2])

In [112]:
consulta_df = pnd.DataFrame()

consulta_df['Consulta'] = consultas
consulta_df['Binario'] = top5_binario
consulta_df['TF'] = top5_tf
consulta_df['TF-IDF'] = top5_tfidf
consulta_df['BM25'] = top5_bm25

consulta_df.index+=1
consulta_df

,Consulta,Binario,TF,TF-IDF,BM25
1,economia,"[(1, 248), (1, 240), (1, 236), (1, 235), (1, 2...","[(7, 138), (7, 125), (6, 34), (5, 127), (4, 26)]","[(11.41, 138), (11.41, 125), (9.78, 34), (8.15...","[(3.85, 138), (3.85, 125), (3.43, 34), (2.97, ..."
2,neymar,"[(1, 121), (1, 74), (1, 45), (0, 249), (0, 248)]","[(2, 121), (1, 74), (1, 45), (0, 249), (0, 248)]","[(8.84, 121), (4.42, 74), (4.42, 45), (0.0, 24...","[(3.67, 121), (1.92, 74), (1.92, 45), (0, 249)..."
3,bolsonaro,"[(1, 248), (1, 240), (1, 238), (1, 237), (1, 2...","[(32, 151), (30, 207), (30, 166), (19, 19), (1...","[(42.24, 151), (39.6, 207), (39.6, 166), (25.0...","[(7.39, 151), (7.21, 207), (7.21, 166), (5.85,..."


## 5. Compare os resultados encontrados e responda.

### 5.1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.

In [113]:
def get_pontuacao_top1(top5):
  return [top[0][0] for top in top5]

def get_documento_top1(top5):
  return [top[0][1] for top in top5]

def get_titulo_documento(docs):
  return [resultado.title[doc] for doc in docs]

In [114]:
pontuacao_binario_top5 = get_pontuacao_top1(top5_binario)
pontuacao_tf_top5 = get_pontuacao_top1(top5_tf)
pontuacao_tf_idf_top5 = get_pontuacao_top1(top5_tfidf)
pontuacao_bm25_top5 = get_pontuacao_top1(top5_bm25)

documento_top5_binario = get_documento_top1(top5_binario)
documento_top5_tf = get_documento_top1(top5_tf)
documento_top5_tf_idf = get_documento_top1(top5_tfidf)
documento_top5_bm25 = get_documento_top1(top5_bm25)

titulos_binario = get_titulo_documento(documento_top5_binario)
titulos_tf = get_titulo_documento(documento_top5_tf)
titulos_tf_idf = get_titulo_documento(documento_top5_tf_idf)
titulos_bm25 = get_titulo_documento(documento_top5_bm25)

#### Representação Binária

In [115]:
rep_binaria = pnd.DataFrame()
rep_binaria['Consulta'] = consultas
rep_binaria['Documento'] = documento_top5_binario
rep_binaria['Titulo Documento'] = titulos_binario
rep_binaria['Pontuacao Binaria'] = pontuacao_binario_top5

rep_binaria.index += 1
rep_binaria

,Consulta,Documento,Titulo Documento,Pontuacao Binaria
1,economia,248,As três espanholas do Estado Islâmico: “Só que...,1
2,neymar,121,“Bonucci diz o que muita gente pensa: os negro...,1
3,bolsonaro,248,As três espanholas do Estado Islâmico: “Só que...,1


#### TF

In [116]:
rep_tf = pnd.DataFrame()
rep_tf['Consulta'] = consultas
rep_tf['Documento'] = documento_top5_tf
rep_tf['Titulo Documento'] = titulos_tf
rep_tf['TF Pontuacao'] = pontuacao_tf_top5

rep_tf.index += 1
rep_tf

,Consulta,Documento,Titulo Documento,TF Pontuacao
1,economia,138,Estatal espanhola fica com aeroportos do Norde...,7
2,neymar,121,“Bonucci diz o que muita gente pensa: os negro...,2
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,32


#### TF-IDF

In [117]:
rep_tf_idf = pnd.DataFrame()
rep_tf_idf['Consulta'] = consultas
rep_tf_idf['Documento'] = documento_top5_tf_idf
rep_tf_idf['Titulo Documento'] = titulos_tf_idf
rep_tf_idf['TFIDF Pontuacao'] = pontuacao_tf_idf_top5

rep_tf_idf.index += 1
rep_tf_idf

,Consulta,Documento,Titulo Documento,TFIDF Pontuacao
1,economia,138,Estatal espanhola fica com aeroportos do Norde...,11.41
2,neymar,121,“Bonucci diz o que muita gente pensa: os negro...,8.84
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,42.24


#### BM-25

In [118]:
rep_bm25 = pnd.DataFrame()
rep_bm25['Consulta'] = consultas
rep_bm25['Documento'] = documento_top5_bm25
rep_bm25['Titulo Documento'] = titulos_bm25
rep_bm25['BM25 Pontuacao'] = pontuacao_bm25_top5

rep_bm25.index += 1
rep_bm25

,Consulta,Documento,Titulo Documento,BM25 Pontuacao
1,economia,138,Estatal espanhola fica com aeroportos do Norde...,3.85
2,neymar,121,“Bonucci diz o que muita gente pensa: os negro...,3.67
3,bolsonaro,151,Socialismo ‘millennial’ nos EUA,7.39


Como podemos ver, TF-IDF e BM25 mostraram resultados similares, esses também foram os melhores.

Para 'economia', obtivemos:
'Estatal espanhola fica com aeroportos do Nordeste em leilão positivo para Bolsonaro'

Para 'neymar', obtivemos:
'Bonucci diz o que muita gente pensa: os negros merecem o que acontece com eles'

Para 'bolsonaro', obtivemos:
'Socialismo ‘millennial’ nos EUA,A temida palavra ‘socialista’ já não é mais tabu entre os jovens do país.'


TF-IDF mostrou melhores resultados.

### 5.2. Calcule e reporte o overlap par-a-par entre os resultados de cada modelo (usando o índice de Jaccard)

In [119]:
def inter_size(a,b):
  return len([elem for elem in a if elem in b])

def indice_jaccard(a,b):
  n_a = len(a)
  n_b = len(b)
  n_ab = inter_size(a,b)
  exp = n_a + n_b - n_ab
  jaccard = n_ab / exp if exp != 0 else 0
  return jaccard

In [120]:
todos_documentos = []
todos_documentos.append(documento_top5_binario)
todos_documentos.append(documento_top5_bm25)
todos_documentos.append(documento_top5_tf)
todos_documentos.append(documento_top5_tf_idf)

matriz = [['Measure', 'Binary', 'TF', 'TF-IDF', 'BM25']]

for i in range(len(todos_documentos)):
  line = []
  for j in range(len(todos_documentos)):
    jaccard = indice_jaccard(todos_documentos[i], todos_documentos[j])
    line.append(round(jaccard,2))
  matriz.append(line)
for i in range(1, len(matriz)):
  matriz[i].insert(0, matriz[0][i])
  
pnd.DataFrame(matriz)

,0,1,2,3,4
0,Measure,Binary,TF,TF-IDF,BM25
1,Binary,1,0.2,0.2,0.2
2,TF,0.2,1,1,1
3,TF-IDF,0.2,1,1,1
4,BM25,0.2,1,1,1
